In [1]:
from web3 import Web3
from solcx import compile_files
import json

In [2]:
# Connect to an ethereum provider via the http
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
print(w3.isConnected())

True


In [3]:
# Test block intended to show that everything works as expected
tx = {
    "from": w3.eth.accounts[1],
    "to": w3.eth.accounts[0],
    "value": w3.toWei(0.5, 'ether')
}
gas = w3.eth.estimateGas(tx)
tx_hash = w3.eth.send_transaction(tx)
w3.eth.waitForTransactionReceipt(tx_hash)
w3.eth.getTransaction(tx_hash)

AttributeDict({'hash': HexBytes('0x26a1431f45a838f9562e43fb4b0b6d065b1ce3ffe8d835704242f888744a9384'),
 'nonce': 0,
 'blockHash': HexBytes('0x1482879bb21ea9928f48e648f663df2759f3a8340f2f70b0c2b242f0f5593798'),
 'blockNumber': 1,
 'transactionIndex': 0,
 'from': '0x5E346Cdbf5406ECd95Fd6F6BA448BA30395172d4',
 'to': '0x938024E7021854d91bC7f34311c2c9C60f12104f',
 'value': 500000000000000000,
 'gas': 121000,
 'gasPrice': 20000000000,
 'input': '0x',
 'v': 38,
 'r': HexBytes('0x5dc158aa49a5da617b4183bbbe3a34926e36d8b0ee57200d66215bcbaeb33001'),
 's': HexBytes('0x33d2051996e2ee214e25ed4e82d158e2468c497305496f89ae14247616a26352')})

In [4]:
# In this block we are creating and deploying a contract from .sol file
def deploy_contract(contract_interface):
    Wallet = w3.eth.contract(
        abi=contract_interface['abi'],
        bytecode=contract_interface['bin']
    )
    w3.eth.defaultAccount = w3.eth.accounts[0]
    tx_hash = Wallet.constructor().transact()
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt['contractAddress']

compiled_sol = compile_files(['contracts/wallet.sol'], output_values=["abi", "bin"])
_, contract_interface = compiled_sol.popitem()

with open('eth_MyWallet.json', 'w') as outfile:
    data = {
        "abi": contract_interface['abi'],
        "contract_address": deploy_contract(contract_interface)
    }
    json.dump(data, outfile)

In [5]:
# In this block we read the data from saved contract
with open('eth_MyWallet.json', 'r') as infile:
    data = json.load(infile)
    contract = w3.eth.contract(abi=data['abi'], address=data['contract_address'])

In [6]:
#-----------------------------------------------
# Test outer transfers
#-----------------------------------------------
w3.eth.defaultAccount = w3.eth.accounts[1]
user_balance = []

#-----------------------------------------------
# Send ether to the Wallet
#-----------------------------------------------
user_balance.append(w3.fromWei(w3.eth.getBalance(w3.eth.accounts[1]), 'ether'))
tx_hash = w3.eth.send_transaction({"to": contract.address, "value": w3.toWei(10, 'ether')})
w3.eth.waitForTransactionReceipt(tx_hash)
print("User's total balance =", w3.fromWei(contract.functions.getBalance(w3.eth.accounts[1]).call(), 'ether'))

#-----------------------------------------------
# Extract ether from wallet
#-----------------------------------------------
owner_balance = contract.functions.getBalance(w3.eth.accounts[0]).call()
user_balance.append(w3.fromWei(w3.eth.getBalance(w3.eth.accounts[1]), 'ether'))

# Owner gets 0.1 ether fee
tx_hash = contract.functions.withdraw(w3.eth.accounts[1], w3.toWei(5, 'ether')).transact()
w3.eth.waitForTransactionReceipt(tx_hash)

user_balance.append(w3.fromWei(w3.eth.getBalance(w3.eth.accounts[1]), 'ether'))
owner_fee = contract.functions.getBalance(w3.eth.accounts[0]).call() - owner_balance

# Sum up the results
print("Owners fee =", w3.fromWei(owner_fee, 'ether'), "ether")
print("Owners balance =", w3.fromWei(contract.functions.getBalance(w3.eth.accounts[0]).call(), 'ether'))
print(f'Transaction history: {user_balance[0]} -> -{user_balance[0] - user_balance[1]} \
-> +{user_balance[2] - user_balance[1]} = {user_balance[2]}')
print(f'Ether loss = {user_balance[0] - user_balance[2] - 5}')

User's total balance = 10
Owners fee = 0.1 ether
Owners balance = 0.1
Transaction history: 99.49958 -> -10.00084328 -> +4.89874078 = 94.3974775
Ether loss = 0.1021025


In [7]:
#-----------------------------------------------
# Test inner transfers
#-----------------------------------------------
user_balance = w3.fromWei(w3.eth.getBalance(w3.eth.accounts[1]), 'ether')

# Send 1 ether to owner
tx_hash = contract.functions.transferTokens(w3.eth.accounts[0], w3.toWei(1, 'ether')).transact()
w3.eth.waitForTransactionReceipt(tx_hash)

print("Users balance = ", w3.fromWei(contract.functions.getBalance(w3.eth.accounts[1]).call(), 'ether'))
print("Users actual balance loss =", user_balance - w3.fromWei(w3.eth.getBalance(w3.eth.accounts[1]), 'ether'))
print("Owners balance =", w3.fromWei(contract.functions.getBalance(w3.eth.accounts[0]).call(), 'ether'))

Users balance =  4
Users actual balance loss = 0.00074698
Owners balance = 1.1


In [8]:
#-----------------------------------------------
# Test allowance
#-----------------------------------------------
w3.eth.defaultAccount = w3.eth.accounts[0]

# Allow w3.eth.accounts[1] use 1 ether
tx_hash = contract.functions.setAllowance(w3.eth.accounts[1], w3.toWei(1, 'ether')).transact()
w3.eth.waitForTransactionReceipt(tx_hash)

w3.eth.defaultAccount = w3.eth.accounts[1]
loaners_balance = w3.fromWei(contract.functions.getBalance(w3.eth.accounts[0]).call(), 'ether')
users_balance = w3.fromWei(contract.functions.getBalance(w3.eth.accounts[1]).call(), 'ether')
print(f'Initial balances: loaner = {loaners_balance}, user = {users_balance}')
allowance = w3.fromWei(contract.functions.getAllowance(w3.eth.accounts[0]).call(), 'ether')

# Send this ether back to acc1
tx_hash = contract.functions.transferTokensFromTo(w3.eth.accounts[0], w3.eth.accounts[1], w3.toWei(1, 'ether')).transact()
w3.eth.waitForTransactionReceipt(tx_hash)

# Count revenue
loaners_balance -= w3.fromWei(contract.functions.getBalance(w3.eth.accounts[0]).call(), 'ether')
users_balance = w3.fromWei(contract.functions.getBalance(w3.eth.accounts[1]).call(), 'ether') - users_balance

print(f'Allowance from acc0 to acc1 = {allowance} ether')
print(f'acc0 revenue = -{loaners_balance} ether')
print(f'acc1 revenue = +{users_balance} ether')

Initial balances: loaner = 1.1, user = 4
Allowance from acc0 to acc1 = 1 ether
acc0 revenue = -1.0 ether
acc1 revenue = +1 ether
